# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [30]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [31]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [32]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [33]:
!mkdir data

mkdir: data: File exists


In [34]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31392    0 31392    0     0   493k      0 --:--:-- --:--:-- --:--:--  494k


In [35]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70251    0 70251    0     0  1119k      0 --:--:-- --:--:-- --:--:-- 1124k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [36]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [37]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [38]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [39]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [40]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 66)

We can save and load our knowledge graphs as follows.

In [41]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 66)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [42]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [43]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


#### Answer
SingleHopSpecificQuerySynthesizer creates straightforward, direct questions that can be answered by a single piece of information from the text.

MultiHopAbstractQuerySynthesizer genertes questions that require combining multiple pieces of information and abstract thinking.

MultiHopSpecificQuerySynthesizer creates questions that need multiple specific pieces of information to be connected.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [44]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,what we learn about llms in 2024?,[Code may be the best application The ethics o...,The most surprising thing we’ve learned about ...,single_hop_specifc_query_synthesizer
1,How do LLMs handle the complexity of languages...,[Based Development As a computer scientist and...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,LLMs easy to make?,[Simon Willison’s Weblog Subscribe Stuff we fi...,Large Language Models (LLMs) are actually quit...,single_hop_specifc_query_synthesizer
3,"Wut iz the impact of ChatGPT on human society,...",[easy to follow. The rest of the document incl...,The impact of ChatGPT on human society is alre...,single_hop_specifc_query_synthesizer
4,What are the notable features and capabilities...,[Voice and live camera mode are science fictio...,"Google's Gemini 1.5 Pro, released in February,...",single_hop_specifc_query_synthesizer
5,How do Large Language Models (LLMs) function a...,[<1-hop>\n\nCode may be the best application T...,Large Language Models (LLMs) function as black...,multi_hop_abstract_query_synthesizer
6,How do the ethics of training data impact the ...,[<1-hop>\n\nCode may be the best application T...,The ethics of training data significantly impa...,multi_hop_abstract_query_synthesizer
7,How do Large Language Models (LLMs) function a...,[<1-hop>\n\nCode may be the best application T...,Large Language Models (LLMs) function as black...,multi_hop_abstract_query_synthesizer
8,How has the development and implementation of ...,"[<1-hop>\n\ntokens, making it easier for a mod...",The development and implementation of Large La...,multi_hop_specific_query_synthesizer
9,How did the development of Claude 3 and the ad...,[<1-hop>\n\nVoice and live camera mode are sci...,The development of Claude 3 by Anthropic and t...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [45]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [46]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What advancements has Mistral made in the fiel...,[Code may be the best application The ethics o...,The team behind Mistral has been working to be...,single_hop_specifc_query_synthesizer
1,Does ChatGPT's performance change during Decem...,[Based Development As a computer scientist and...,The honest answer is 'maybe'! No-one is entire...,single_hop_specifc_query_synthesizer
2,What were the significant developments in LLMs...,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,Wht is the Plausible analytics data showing ab...,[easy to follow. The rest of the document incl...,The Plausible analytics data shows that AI-rel...,single_hop_specifc_query_synthesizer
4,What are the ethical concerns associated with ...,[<1-hop>\n\nCode may be the best application T...,The ethical concerns associated with the train...,multi_hop_abstract_query_synthesizer
5,What are the challenges and advancements in un...,[<1-hop>\n\nCode may be the best application T...,Large Language Models (LLMs) present significa...,multi_hop_abstract_query_synthesizer
6,How do the challenges of LLMs being black boxe...,[<1-hop>\n\nCode may be the best application T...,The development and use of Large Language Mode...,multi_hop_abstract_query_synthesizer
7,How do the challenges of LLMs as black boxes i...,[<1-hop>\n\nCode may be the best application T...,The challenges of LLMs as black boxes signific...,multi_hop_abstract_query_synthesizer
8,How does the cost efficiency of GPT-4o compare...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,GPT-4o is significantly more cost-efficient th...,multi_hop_specific_query_synthesizer
9,How has Meta's approach to training data and m...,"[<1-hop>\n\ntokens, making it easier for a mod...",Meta's approach to training data and model fin...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [49]:
from langsmith import Client
import uuid

client = Client()

dataset_name = f"State of AI Across the Years! {uuid.uuid4().hex[:8]}"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
) 

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [50]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [51]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [52]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [53]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [54]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [55]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [56]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [57]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [58]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [59]:
rag_chain.invoke({"question" : "What are Agents?"})

'The term "agents" in the context provided is described as an infuriatingly vague term that seems to converge on "AI systems that can go away and act on your behalf." There are two main categories of thought regarding agents: one views them as things that act on one\'s behalf similar to a travel agent, while the other thinks of them as LLMs (large language models) that have been given access to tools to solve problems. However, the term lacks a single, clear definition, and the concept of agents is still largely theoretical without many real-world examples of them running in production.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [60]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [61]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

#### Answer
### qa_evaluator :
    * Evaluates the correctness and accuracy of the answers compared to the reference answers
    * Focuses on factual accuracy and completeness

### labeled_helpfulness_evaluator :
    * Assesses how helpful ht eresponse is to the user
    * Considers both accuracy and usefulness while comparing against the reference answer
    * Takes into account the original question context

### dope_or_nope_evaluator :
    * Evaluates the style and engagement level of the response
    * Checks if the answer is presented in an engaging, cool, or interesting way
    * Focuses on the presentation rather than just accuracy

## LangSmith Evaluation

In [62]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'warm-powder-92' at:
https://smith.langchain.com/o/e60ce1c7-6903-49a6-9264-5c7bd472b609/datasets/39047d6c-80af-4883-99eb-7b815fca43c1/compare?selectedSessions=fa0d4559-983b-4cc3-b7ec-9b2d7cddb537




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How has Anthropic's Claude Artifacts feature i...,Anthropic's Claude Artifacts feature has signi...,None,Anthropic's Claude Artifacts feature has signi...,1,0,1,1.986254,b0450a0a-be6c-413e-9094-b67879a73ab7,d990f66c-0d8e-4164-971d-7ce8f00dd5ee
1,How did the introduction of prompt-driven app ...,The introduction of prompt-driven app generati...,None,The introduction of prompt-driven app generati...,0,0,0,2.048627,c38a360b-944c-4436-aa37-bee574df2ea8,d7dd7875-5812-4553-b860-a6af90daa9d8
2,How has Meta's approach to training data and m...,Meta's approach to training data and model fin...,None,Meta's approach to training data and model fin...,1,0,1,3.060095,bf892012-4fe1-423c-8d8b-378746734aa6,2b4ba4cf-aa7d-44d4-8029-94b90b8ed1cf
3,How does the cost efficiency of GPT-4o compare...,The cost efficiency of GPT-4o is significantly...,None,GPT-4o is significantly more cost-efficient th...,1,1,1,4.129695,85934f14-f3cf-40d3-9e23-c9eb0863ddb1,a29f8b0c-a196-4369-ae02-b64f4870558c
4,How do the challenges of LLMs as black boxes i...,The challenges of LLMs as black boxes signific...,None,The challenges of LLMs as black boxes signific...,1,1,0,6.010494,435dc083-f65f-489a-bc48-a1b5bf5377a2,1b0b1dfe-36f6-4f13-8047-80f5ba70b014
5,How do the challenges of LLMs being black boxe...,I don't know.,None,The development and use of Large Language Mode...,0,0,0,0.796280,1a9fe615-8259-47a0-afe9-1db0cab8045f,50a017a3-14ee-4c03-8f81-436e02f24cf3
6,What are the challenges and advancements in un...,I don't know.,None,Large Language Models (LLMs) present significa...,0,0,0,1.053976,5b7215c4-0f76-499d-be4f-5a490a99a196,45b7df47-00c1-4bb6-a372-bf0e56641716
7,What are the ethical concerns associated with ...,The ethical concerns associated with the train...,None,The ethical concerns associated with the train...,1,0,0,3.582046,ec837b8d-cd9f-4942-a046-5ba0729bec30,ae912461-a388-428d-ad2f-575218a407b9
8,Wht is the Plausible analytics data showing ab...,The Plausible analytics data shows that the to...,None,The Plausible analytics data shows that AI-rel...,0,0,0,1.727137,fd5394bc-51d3-480e-99bb-bec53fda7ff7,556bb505-953f-453a-893d-695c116b983d
9,What were the significant developments in LLMs...,I don't know.,None,2023 was the breakthrough year for Large Langu...,0,0,0,2.951873,3887044f-ede3-479d-b0b4-ded91439fb8b,e4ca374b-7e66-4e7c-a8a5-0e8988718609


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [63]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [64]:
rag_documents = docs

In [65]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### Answer
Modifying chunk size provides more context in each segment which helps to maintain coherence and relationships between ideas. However, larger chunks might also include irrelevant information. The trade-off is between context completeness and precision in retrieval. Optimal chunk size depends on the nature of the content and the types of questions being asked.

In [66]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### Answer

Changing from text-embedding-3-small to text-embedding-3-large affects performance because:

* Larger embedding models create more detailed vector representations
* Better captures nuanced semantic relationships in the text
* Improves similarity matching during retrieval
* Can better understand context and subtle differences
* Results in more accurate document retrieval for complex queries

In [67]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [68]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [69]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [70]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Yo, agents are a super fuzzy concept in the AI world, man! They\'re basically described as "AI systems that can act on your behalf," like a travel agent or some high-tech assistant. But here\'s the kicker—there\'s no solid definition everyone agrees on. Some think of them as LLMs using tools to solve problems, while others just see them as digital helpers. But right now, they\'re still kinda in the "coming soon" phase, mostly because of the gullibility issue. Basically, if the AI can\'t tell truth from fiction, how effective can it really be? So, yeah, the whole agent scene is a bit of a mystery wrapped in an enigma, fam!'

Finally, we can evaluate the new chain on the same test set!

In [71]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'scholarly-vegetable-61' at:
https://smith.langchain.com/o/e60ce1c7-6903-49a6-9264-5c7bd472b609/datasets/39047d6c-80af-4883-99eb-7b815fca43c1/compare?selectedSessions=0fa9b30b-7a89-419b-b472-85a5cb375e78




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How has Anthropic's Claude Artifacts feature i...,"Yo, Claude Artifacts by Anthropic dropped a bo...",None,Anthropic's Claude Artifacts feature has signi...,1,1,1,2.431130,b0450a0a-be6c-413e-9094-b67879a73ab7,1a21abc1-e611-42bc-9407-684a2a32e42c
1,How did the introduction of prompt-driven app ...,"Yo, check it! The whole prompt-driven app gene...",None,The introduction of prompt-driven app generati...,0,0,1,2.675457,c38a360b-944c-4436-aa37-bee574df2ea8,8eff4d1d-5468-4f2b-9204-5edbfc3fc7a5
2,How has Meta's approach to training data and m...,"Yo, here's the lowdown on Meta's vibe with LLM...",None,Meta's approach to training data and model fin...,1,0,1,5.284265,bf892012-4fe1-423c-8d8b-378746734aa6,c5e635c8-a5be-4564-a195-0e966ef6338a
3,How does the cost efficiency of GPT-4o compare...,"Yo, check it! GPT-4o is going for just $2.50, ...",None,GPT-4o is significantly more cost-efficient th...,1,1,1,2.425518,85934f14-f3cf-40d3-9e23-c9eb0863ddb1,27079390-8c8e-4a4d-8656-7438f6d04944
4,How do the challenges of LLMs as black boxes i...,"Yo, the challenges of LLMs being these mysteri...",None,The challenges of LLMs as black boxes signific...,1,0,1,5.534364,435dc083-f65f-489a-bc48-a1b5bf5377a2,c3ea3715-587e-4caf-93cb-c9bb556d6d5b
5,How do the challenges of LLMs being black boxe...,"Yo, check it out! The challenges of LLMs being...",None,The development and use of Large Language Mode...,1,0,1,3.039999,1a9fe615-8259-47a0-afe9-1db0cab8045f,e712c8da-14ae-4377-a32d-2c5fa1d17319
6,What are the challenges and advancements in un...,"Yo, let’s break it down! The challenges with L...",None,Large Language Models (LLMs) present significa...,1,0,1,3.491108,5b7215c4-0f76-499d-be4f-5a490a99a196,79cbf68d-38ce-4c07-8de3-d3fc0be792f2
7,What are the ethical concerns associated with ...,"Yo, the whole scene around Large Language Mode...",None,The ethical concerns associated with the train...,1,1,1,2.954705,ec837b8d-cd9f-4942-a046-5ba0729bec30,70d3e2c4-ae20-4cc3-aeb7-6f8186c348d2
8,Wht is the Plausible analytics data showing ab...,"Yo, the Plausible analytics data is showcasing...",None,The Plausible analytics data shows that AI-rel...,0,0,1,1.857442,fd5394bc-51d3-480e-99bb-bec53fda7ff7,66455fb8-3d5a-4a08-83db-1973452efd48
9,What were the significant developments in LLMs...,"Yo, the context doesn't really spill the tea o...",None,2023 was the breakthrough year for Large Langu...,0,0,0,1.334527,3887044f-ede3-479d-b0b4-ded91439fb8b,495fbca0-2787-4dff-bd36-360afbecbe73


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.


![Differences](./data/Screenshot%202025-02-17%20at%201.07.16 PM.png)

#### Answer
Changes between the two chains:

* QA accuracy improved due to better context retrieval due to better embedding model.
* Helpfulness scores increased due to more comprehensive context
* "Dopeness" scores increased due to the explicit prompt modification

Overall performance improvement from the combination of:

* Larger chunk size providing more context
* Better embedding model improving retrieval
* "Dope" prompt enhancement making responses more engaging